In [1]:
library(tidyverse)
library(lfe)
library(data.table)
library(dplyr)
library(stargazer)
library(readxl)
library(tidyr)
library(MASS)

Warning message:
“package ‘ggplot2’ was built under R version 4.3.1”
Warning message:
“package ‘tidyr’ was built under R version 4.3.1”
Warning message:
“package ‘readr’ was built under R version 4.3.1”
Warning message:
“package ‘dplyr’ was built under R version 4.3.1”
Warning message:
“package ‘lubridate’ was built under R version 4.3.1”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
“package ‘lfe’ was built under R version 4.3.3”
Loading required package: Matrix


Attaching package: ‘Matrix’


The f

In [2]:
df_gains <- read_excel("../data/GAINS/GHGemis_vs_PM25_conc_GAINS_countries_regions_v2.xlsx", sheet = "Export Worksheet")

In [3]:
df2 <- df_gains %>%
    mutate(
        logpm = log(PM25_ANTHROP),
        loggdp = log(GDP_PPP),
        logpop = log(POP),
        loggdppc = log(GDP_PPP / POP),
        loggdppc2 = loggdppc^2,
        logco2 = log(EMIS_CO2_KT),
        logch4 = log(EMIS_CH4_KT)
    ) %>%
    group_by(REGION_4LETTER, IDSCENARIOS) %>%
    arrange(IDYEARS) %>%
    mutate(
        logpm0 = logpm - first(logpm),
        loggdp0 = loggdp - first(loggdp),
        logpop0 = logpop - first(logpop),
        loggdppc0 = loggdppc - first(loggdppc),
        loggdppc02 = loggdppc2 - first(loggdppc2),
        logco20 = logco2 - first(logco2),
        logch40 = logch4 - first(logch4),
        year0 = IDYEARS - first(IDYEARS),
        laglogpm0 = lag(logpm0),
        lag2logpm0 = lag(logpm0, 2),
        logco20xyear0 = logco20 * year0,
        logch40xyear0 = logch40 * year0
    ) %>%
    ungroup()

# Replace NA values with 0
df2 <- df2 %>%
    mutate(
        laglogpm0 = ifelse(is.na(laglogpm0), 0, laglogpm0),
        lag2logpm0 = ifelse(is.na(lag2logpm0), 0, lag2logpm0)
    )

## PM2.5 ANTHOROPOGENIC

In [4]:
summary(
    model1 <- felm(logpm0 ~ logco20 + logch40 + logco20xyear0 + logch40xyear0  + logpop0 + 
               loggdppc0 + loggdppc02 + laglogpm0 + lag2logpm0 | factor(IDYEARS), 
               data = subset(df2, IDSCENARIOS == "Decarb"))
        )


Call:
   felm(formula = logpm0 ~ logco20 + logch40 + logco20xyear0 + logch40xyear0 +      logpop0 + loggdppc0 + loggdppc02 + laglogpm0 + lag2logpm0 |      factor(IDYEARS), data = subset(df2, IDSCENARIOS == "Decarb")) 

Residuals:
     Min       1Q   Median       3Q      Max 
-0.95632 -0.01402  0.00000  0.02662  0.20767 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
logco20        0.036792   0.037012   0.994  0.32071    
logch40       -0.070948   0.055299  -1.283  0.20013    
logco20xyear0 -0.001770   0.002176  -0.813  0.41653    
logch40xyear0  0.003015   0.003239   0.931  0.35239    
logpop0        0.201869   0.055281   3.652  0.00029 ***
loggdppc0      0.139119   0.160604   0.866  0.38681    
loggdppc02     0.005812   0.007618   0.763  0.44589    
laglogpm0      1.376500   0.046567  29.559  < 2e-16 ***
lag2logpm0    -0.405954   0.063188  -6.425 3.24e-10 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.07459 o

## OUT-OF-COUNTRY CONTRIBUTION

In [5]:
file_path <- "../data/GAINS/PM25_sourcecontrib_GAINS_4letter.xlsx"
df_export <- read_excel(file_path, sheet = "Export Worksheet")
df_long <- df_export %>%
    pivot_longer(cols = -c(IDSCENARIOS, IDYEARS, RECEPTOR_4LETTER),
                 names_to = "Source",
                 values_to = "PM25_Contribution") %>%
    rename(REGION_4LETTER = RECEPTOR_4LETTER)

df_agg_split <- df_long %>%
    group_by(IDSCENARIOS, IDYEARS, REGION_4LETTER) %>%
    summarise(
        PM25_TOTAL = sum(PM25_Contribution, na.rm = TRUE),
        PM25_OUT   = sum(PM25_Contribution[REGION_4LETTER != Source], na.rm = TRUE),
        PM25_SELF  = sum(PM25_Contribution[REGION_4LETTER == Source], na.rm = TRUE),
        .groups = "drop"
    )

df_agg_export <- df_long %>%
    filter(Source != REGION_4LETTER) %>%  # Exclude self-contributions
    group_by(IDSCENARIOS, IDYEARS, Source) %>%
    summarise(
        PM25_EXPORT = sum(PM25_Contribution, na.rm = TRUE),
        .groups = "drop"
    )

df2_agg <- df_agg_split %>%
    left_join(df_agg_export, by = c("IDSCENARIOS", "IDYEARS", "REGION_4LETTER" = "Source"))

df2 <- df2 %>%
    left_join(df2_agg, by = c("IDSCENARIOS", "IDYEARS", "REGION_4LETTER"))

In [6]:
df2_out <- df2 %>%
    mutate(
        logpm = log(PM25_OUT)  # Changed to PM25_OUT

    ) %>%
    group_by(REGION_4LETTER, IDSCENARIOS) %>%
    arrange(IDYEARS) %>%
    mutate(
        logpm0 = logpm - first(logpm),
        laglogpm0 = lag(logpm0),
        lag2logpm0 = lag(logpm0, 2)
    ) %>%
    ungroup()
df2_out <- df2_out %>%
    mutate(
        logpm = ifelse(is.infinite(logpm) | is.na(logpm), 0, logpm),
        laglogpm0 = ifelse(is.na(laglogpm0), 0, laglogpm0),
        lag2logpm0 = ifelse(is.na(lag2logpm0), 0, lag2logpm0)
    )


In [7]:
summary(
    model2 <- felm(logpm0 ~ logco20 + logch40 + logco20xyear0 + logch40xyear0  + logpop0 + 
               loggdppc0 + loggdppc02 + laglogpm0 + lag2logpm0 | factor(IDYEARS), 
               data = subset(df2_out, IDSCENARIOS == "Decarb"))
        )


Call:
   felm(formula = logpm0 ~ logco20 + logch40 + logco20xyear0 + logch40xyear0 +      logpop0 + loggdppc0 + loggdppc02 + laglogpm0 + lag2logpm0 |      factor(IDYEARS), data = subset(df2_out, IDSCENARIOS == "Decarb")) 

Residuals:
      Min        1Q    Median        3Q       Max 
-0.143898 -0.003759  0.000000  0.004324  0.071784 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
logco20        0.0467961  0.0121945   3.837 0.000141 ***
logch40       -0.0139748  0.0183864  -0.760 0.447598    
logco20xyear0 -0.0025725  0.0007178  -3.584 0.000374 ***
logch40xyear0  0.0005577  0.0010778   0.517 0.605053    
logpop0        0.0311814  0.0193942   1.608 0.108556    
loggdppc0      0.0225930  0.0526658   0.429 0.668128    
loggdppc02     0.0008482  0.0024682   0.344 0.731279    
laglogpm0      1.6102482  0.0442740  36.370  < 2e-16 ***
lag2logpm0    -0.6235006  0.0639557  -9.749  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual s

## SELF CONTRIBUTION

In [8]:
df2_self <- df2 %>%
    mutate(
        logpm = log(PM25_SELF)  # Changed to PM25_SELF

    ) %>%
    group_by(REGION_4LETTER, IDSCENARIOS) %>%
    arrange(IDYEARS) %>%
    mutate(
        logpm0 = logpm - first(logpm),
        laglogpm0 = lag(logpm0),
        lag2logpm0 = lag(logpm0, 2)
    ) %>%
    ungroup()
df2_self <- df2_self %>%
    mutate(
        logpm = ifelse(is.infinite(logpm) | is.na(logpm), 0, logpm),
        laglogpm0 = ifelse(is.na(laglogpm0), 0, laglogpm0),
        lag2logpm0 = ifelse(is.na(lag2logpm0), 0, lag2logpm0)
    )


In [9]:
summary(
    model3 <- felm(logpm0 ~ logco20 + logch40 + logco20xyear0 + logch40xyear0  + logpop0 + 
               loggdppc0 + loggdppc02 + laglogpm0 + lag2logpm0 | factor(IDYEARS), 
               data = subset(df2_self, IDSCENARIOS == "Decarb"))
        )


Call:
   felm(formula = logpm0 ~ logco20 + logch40 + logco20xyear0 + logch40xyear0 +      logpop0 + loggdppc0 + loggdppc02 + laglogpm0 + lag2logpm0 |      factor(IDYEARS), data = subset(df2_self, IDSCENARIOS == "Decarb")) 

Residuals:
     Min       1Q   Median       3Q      Max 
-0.92257 -0.01746  0.00000  0.04684  0.26383 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
logco20        0.067659   0.051654   1.310    0.191    
logch40       -0.039782   0.078424  -0.507    0.612    
logco20xyear0 -0.003873   0.003043  -1.273    0.204    
logch40xyear0  0.001338   0.004617   0.290    0.772    
logpop0        0.356038   0.079785   4.462 1.01e-05 ***
loggdppc0      0.270791   0.224835   1.204    0.229    
loggdppc02     0.009978   0.010623   0.939    0.348    
laglogpm0      1.422784   0.047136  30.185  < 2e-16 ***
lag2logpm0    -0.457356   0.065748  -6.956 1.17e-11 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.10

## EXPORT CONTRIBUTION

In [10]:
df2_export <- df2 %>%
    mutate(
        logpm = log(PM25_EXPORT)  # Changed to PM25_EXPORT

    ) %>%
    group_by(REGION_4LETTER, IDSCENARIOS) %>%
    arrange(IDYEARS) %>%
    mutate(
        logpm0 = logpm - first(logpm),
        laglogpm0 = lag(logpm0),
        lag2logpm0 = lag(logpm0, 2)
    ) %>%
    ungroup()
df2_export <- df2_export %>%
    mutate(
        logpm = ifelse(is.infinite(logpm) | is.na(logpm), 0, logpm),
        laglogpm0 = ifelse(is.na(laglogpm0), 0, laglogpm0),
        lag2logpm0 = ifelse(is.na(lag2logpm0), 0, lag2logpm0)
    )


In [11]:
df2_export <- df2_export %>%
    drop_na(PM25_EXPORT, EMIS_CO2_KT, EMIS_CH4_KT, GDP_PPP) %>% 
    filter(PM25_EXPORT > 0, EMIS_CO2_KT > 0, EMIS_CH4_KT > 0, GDP_PPP > 0)

In [12]:
summary(
    model4 <- felm(logpm0 ~ logco20 + logch40 + logco20xyear0 + logch40xyear0  + logpop0 + 
               loggdppc0 + loggdppc02 + laglogpm0 + lag2logpm0 | factor(IDYEARS), 
               data = subset(df2_export, IDSCENARIOS == "Decarb"))
        )


Call:
   felm(formula = logpm0 ~ logco20 + logch40 + logco20xyear0 + logch40xyear0 +      logpop0 + loggdppc0 + loggdppc02 + laglogpm0 + lag2logpm0 |      factor(IDYEARS), data = subset(df2_export, IDSCENARIOS ==      "Decarb")) 

Residuals:
     Min       1Q   Median       3Q      Max 
-0.45391 -0.01901  0.00000  0.03441  0.28840 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
logco20        0.093730   0.042244   2.219  0.02699 *  
logch40       -0.169876   0.062659  -2.711  0.00696 ** 
logco20xyear0 -0.005200   0.002474  -2.102  0.03610 *  
logch40xyear0  0.010405   0.003725   2.793  0.00543 ** 
logpop0        0.185792   0.062269   2.984  0.00300 ** 
loggdppc0      0.279153   0.182019   1.534  0.12580    
loggdppc02     0.009700   0.008518   1.139  0.25544    
laglogpm0      1.413266   0.048130  29.364  < 2e-16 ***
lag2logpm0    -0.399084   0.071064  -5.616 3.39e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard erro

##  Only do regressions for _SELF and _EXPORT, use full model for these (9 coeffs)

### _SELF is model3  and _EXPORT is model4

In [32]:
#mvrnorm(1000, coef(model3), vcov(model3), empirical=T)
write.csv(as.data.frame(MASS::mvrnorm(1000, coef(model3), vcov(model3), empirical = TRUE)), "mvrnorm_SELF_Contribution.csv", row.names = FALSE)


In [31]:
getwd()

[1] "/Users/taky/Library/CloudStorage/GoogleDrive-tahmid@udel.edu/.shortcut-targets-by-id/1eGPs0oRCM_msajXTz-8JAX1bzMezBk8J/CCAC/analysis"

In [33]:
#mvrnorm(1000, coef(model4), vcov(model4), empirical=T)
write.csv(as.data.frame(MASS::mvrnorm(1000, coef(model4), vcov(model4), empirical = TRUE)), "mvrnorm_EXPORT_Contribution.csv", row.names = FALSE)
